In [1]:
import os
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC'

In [3]:
import mlflow
from dotenv import load_dotenv
import os

load_dotenv()

mlflow.set_tracking_uri(os.getenv('MLFLOW_URI'))
os.environ["MLFLOW_TRACKING_USERNAME"]=os.getenv('MLFLOW_TRACKING_USERNAME')
os.environ["MLFLOW_TRACKING_PASSWORD"]=os.getenv('MLFLOW_TRACKING_PASSWORD')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model=self.config.model_training.trained_model_path,
            training_data_path=self.config.data_ingestion.data_dir,
            mlflow_uri=self.config.model_evaluation.dagshub_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, TensorSpec

In [25]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        val_data = os.path.join(self.config.training_data_path, "seti-sub\\seti_signals_sub\\valid")
        val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                val_data,
                                shuffle=True,
                                image_size = (self.config.params_image_size[0], self.config.params_image_size[0]),
                                batch_size = self.config.params_batch_size
                            )
        val = val_dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
        
        self.model = self.load_model(self.config.path_of_model)
        self.score = self.model.evaluate(val)
        self.save_score()

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                # Define the input and output schema
                input_schema = Schema([TensorSpec(np.dtype("float32"), (-1, 256, 256, 3))])
                output_schema = Schema([TensorSpec(np.dtype("float32"), (-1, 7))])

                # Create the model signature
                signature = ModelSignature(inputs=input_schema, outputs=output_schema)
                mlflow.keras.log_model(self.model, "model", registered_model_name="InceptionV3", signature=signature)
            else:
                mlflow.keras.log_model(self.model, "model")

NameError: name 'EvaluationConfig' is not defined

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    eval = Evaluation(eval_config)
    eval.evaluation()
    eval.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-03 22:02:30,042: INFO: common: yaml file config\config.yaml loadded successfully]
[2024-07-03 22:02:30,045: INFO: common: yaml file params.yaml loadded successfully]
[2024-07-03 22:02:30,046: INFO: common: Created directory at: ['artifacts']]
Found 175 files belonging to 7 classes.
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 533ms/step - accuracy: 0.1585 - loss: 3783940169515386535936.0000
[2024-07-03 22:02:48,309: INFO: common: Json saved at: scores.json]


2024/07/03 22:02:50 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Registered model 'InceptionV3' already exists. Creating a new version of this model...
2024/07/03 22:04:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: InceptionV3, version 2
Created version '2' of model 'InceptionV3'.


# prediction

In [13]:
from mlflow.models.signature import infer_signature
import os
import tensorflow as tf
import numpy as np
from tensorflow.image import resize

model = tf.keras.models.load_model("D:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC\\artifacts\\model_training\\model.keras")


In [16]:
from tensorflow.keras.preprocessing import image

imagename = "517_brightpixel.png"
test_image = image.load_img(imagename, target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = np.argmax(model.predict(test_image), axis=1)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[2]
